<a id='0'></a>
### 　　目　录
* [第1章 初探大数据](#1)
* [第2章 Spark及其生态圈概述](#2)
* [第3章 实战环境搭建](#3)
* [第4章 Spark SQL概述](#4)
* [第5章 从Hive平滑过渡到Spark SQL](#5)
* [第6章 DateFrame&Dataset](#6)
* [第7章 External Data Source](#7)
* [第8章 SparkSQL愿景](#8)
* [第9章 慕课网日志实战](#9)

<a id='1'></a>
# [第1章 初探大数据](#0)


目录：/home/hadoop
software: 存放安装的软件
app:存放所有软件的安装目录
data：存放课程所有的测试数据目录
source：存放的是软件源码的目录，如spark

groupadd hadoop      创建hadoop用户组  
useradd -m -g hadoop hadoop    新建hadoop用户并增加到hadoop用户组中  

passwd hadoop     hadoop用户密码，为hadoop  
* 修改时区： http://www.cnblogs.com/kerrycode/p/4217995.html

环境参数<img src = 'https://upload-images.jianshu.io/upload_images/6344527-77faf8b26e275559.png?imageMogr2/auto-orient/strip%7CimageView2/2/w/1240' width='66%'>

**software 中的软件**![s**oftware 中的软件**](https://upload-images.jianshu.io/upload_images/6344527-16dc3af5ae916884.png?imageMogr2/auto-orient/strip%7CimageView2/2/w/1240)

## 10 
### 1.hadoop 下载
wget http://archive.cloudera.com/cdh5/cdh/5/hadoop-2.6.0-cdh5.7.0.tar.gz  
tar -zxvf hadoop-2.6.0-cdh5.7.0.tar.gz   -C ~/app/

  
### jdk 下载
wget http://ftp.upf.br/pub/linux/java/jdk-7u51-linux-x64.tar.gz
### 解压
tar -zxvf jdk-7u51-linux-x64.tar.gz -C ~/app/

验证安装

配置到系统**环境变量**

vi  ~/.bash_profile  
source ~/.bash_profile
```
export JAVA_HOME=/home/hadoop/app/jdk1.7.0_51
export PATH=$JAVA_HOME/bin:$PATH

export HADOOP_HOME=/home/hadoop/app/hadoop-2.6.0-cdh5.7.0
export PATH=$HADOOP_HOME/bin:$PATH

export HIVE_HOME=/home/hadoop/app/hive-1.1.0-cdh5.7.0  
export PATH=$HIVE_HOME/bin:$PATH  

export SCALA_HOME=/home/hadoop/app/scala-2.11.8
export PATH=$SCALA_HOME/bin:$PATH 

```
### 3.机器参数设置
    hostname: hekuang
    修改：/etc/sysconfig/network
ubuntu: /etc/network/interfaces.

设置ip和hostname的映射关系：/etc/hosts

### 静态ip
注意：无线连接，可在路由器配置

NM_CONTROLLED=yes
BOOTPROTO=static
IPADDR=192.168.1.106
NETMASK=255.255.255.0
GATEWAY=192.168.1.6


service network restart



### ssh免密码登录 

ssh-keygen  -t rsa　  　  
cp ~/.ssh/id_rsa.pub ~/.ssh/authorized_keys  
再追加到各个主机实现，免密码登录
### 防火墙
https://blog.csdn.net/shuaigexiaobo/article/details/78190168

### hadoop配置文件修改
* bin : 客户端操作的文件
* sbin ： 启动集群  

~/app/hadoop-2.6.0-cdh5.7.0$ cd etc/hadoop

* 修改 hadoop-env.sh

> `export JAVA_HOME=/home/hadoop/app/jdk1.7.0_51`

*  修改 etc/hadoop/core-site.xml

```
 <configuration>  
    <property>
        <name>fs.defaultFS</name>
        <value>hdfs://IP:8020</value>
    </property>

<property>
        <name>hadoop.tmp.dir</name>
        <value>/home/hadoop/app/tmp</value>
    </property>
</configuration>
```

* 修改 hdfs-site.xml
```
<property>
        <name>dfs.replication</name>
        <value>1</value>
    </property>
```
* vi slaves
改为本机ip

* 仅一次格式化 bin/hdfs namenode -format
* 启动 sbin/start-dfs.sh
http://192.168.128.131:50070/

## YARN 产生背景
* MapReduce 1.x 存在问题：单点故障，节点压力大不易扩展  
* JobTracker : 负责资源管理和作业调度
* TaskTracker : 

 YARN : 不同计算框架可以共享同一个HDFS集群上的数据，享受整体的资源调度   
 XXX on YARN   Yet Another Rsesource Negotiator

## YARN 架构
 YARN 架构 
[http://hadoop.apache.org/docs/current/hadoop-yarn/hadoop-yarn-site/YARN.html](http://hadoop.apache.org/docs/current/hadoop-yarn/hadoop-yarn-site/YARN.html)

* ResouceManager ：RM   
   整个集群同一时间提供服务的RM只有**一个**，负责集群资源的统一调度和管理  
  监控NM， 一旦NM挂掉，该NM运行的任务需要告诉我们的AM来如何进行 处理
  处理客户端请求：提交一个作业，杀手一个作业  

* etc/hadoop/mapred-site.xml:
cp mapred-site.xml.template mapred-site.xml

```
<configuration>
    <property>
        <name>mapreduce.framework.name</name>
        <value>yarn</value>
    </property>
</configuration>\
```

* etc/hadoop/yarn-site.xml:


```
<configuration>
    <property>
        <name>yarn.nodemanager.aux-services</name>
        <value>mapreduce_shuffle</value>
    </property>
</configuration>
```
* 启动   $ sbin/start-yarn.sh
* 验证 ResourceManager - http://localhost:8088/
*　提交Mr作业到YARN上运行  
hadoop jar  /home/hadoop/app/hadoop-2.6.0-cdh5.7.0/share/hadoop/mapreduce/hadoop-mapreduce-examples-2.6.0-cdh5.7.0.jar wordcount /input/wc/hello.txt /output/wc/

* 提交作业到yarn
date 中创建txt文件  
hadoop fs -mkdir -p /input/wc   
hadoop fs -put  hello.txt /input/wc/

## Hive

https://cwiki.apache.org/confluence/display/Hive/Home


#### 1.21 Hive产生背景
* MapReduce 编程的不便性
* HDFS 的文件缺少Schema  
##### Hive 是什么  
* Facebook开源，解决海量结构化的日志数据统计问题
* 构建在Hadoop之上的数据仓库
* Hive 定义类一种SQL查询语言：HQL（类似SQL但不完全相同）
* 通常进行离线数据处理（采用MapReduce）
* 底层支持多种不同的执行格式
* 支持多种不同的压缩格式、存储格式以及自定义函数 压缩：GZIP、LZO、Snappy

#### 1.22 
为什么使用Hive
* 简单，易上手
* 为超大数据集设计的计算/存储扩展能力
* **统一的元数据管理（可与Presto/Impala/SparkSQL等共享数据）**

#### 1.23 大数据数据仓库Hive架构以及部署
### [Hive安装配置](http://blog.51cto.com/hatech/1427748)
#### 1-24 -Hive环境搭建
* 下载：
   * wget http://archive.cloudera.com/cdh5/cdh/5/hive-1.1.0-cdh5.7.0.tar.gz  
   * wget http://ftp.ntu.edu.tw/MySQL/Downloads/Connector-J/mysql-connector-java-5.1.46.tar.gz
   解压传入 mysql-connector-java-5.1.46-bin.jar .
   
* 解压：tar -zxvf hive-1.1.0-cdh5.7.0.tar.gz -C ~/app/

* 配置：  
export HIVE_HOME=/home/hadoop/app/hive-1.1.0-cdh5.7.0  
export PATH=\$HIVE_HOME/bin:\$PATH  

* 设置hadoop home
  * /usr/hadoop/conf/ 
  * cp hive-env.sh.template hive-env.sh  
  * vi hive-env.sh
  * HADOOP_HOME=/home/hadoop/app/hadoop-2.6.0-cdh5.7.0



* 本地独立模式
注意：关闭防火墙：

```
# yum install mysql mysql-server          //安装mysql
# service mysqld start
# mysql -u root                           //添加数据库及用户
mysql> create database hive;      
Query OK, 1 row affected (0.00 sec)
mysql> grant all on hive.* to 'hive'@'localhost' identified by 'hive';
Query OK, 0 rows affected (0.00 sec)
mysql> flush privileges;
Query OK, 0 rows affected (0.00 sec)
```





*  拷贝mysql驱动到$HIVE_HOME/lib/  
cp ~/software/mysql-connector-java-5.1.46-bin.jar .
 /home/hadoop/software/mysql-connector-java-5.1.46  

UNIX socket:		/var/lib/mysql/mysql.sock

* 启动hive

/home/hadoop/app/hive-1.1.0-cdh5.7.0/conf









In [ ]:
$ vim /usr/hadoop/conf/hive-site.xml 
<property>
  <name>javax.jdo.option.ConnectionURL</name>         //所连接的MySQL数据库实例
  <value>jdbc:mysql://localhost:3306/sparksql?createDatabaseIfNotExist=true</value>
  <description>JDBC connect string for a JDBC metastore</description>
</property>
 
<property>
  <name>javax.jdo.option.ConnectionDriverName</name>  //连接的MySQL数据库驱动
  <value>com.mysql.jdbc.Driver</value>
  <description>Driver class name for a JDBC metastore</description>
</property>
 
<property>
  <name>javax.jdo.option.ConnectionUserName</name>   //连接的MySQL数据库用户名
  <value>root</value>
  <description>username to use against metastore database</description>
</property>

<property>
  <name>javax.jdo.option.ConnectionPassword</name>    //连接的MySQL数据库密码
  <value>root</value>
  <description>password to use against metastore database</description>
</property>

或者：
<configuration>
<property>
  <name>javax.jdo.option.ConnectionURL</name>         
    <value>jdbc:mysql://localhost:3306/hive</value>
    </property>
     
<property>
  <name>javax.jdo.option.ConnectionDriverName</name>  
    <value>com.mysql.jdbc.Driver</value>
      <description>Driver class name for a JDBC metastore</description>
      </property>
       
<property>
  <name>javax.jdo.option.ConnectionUserName</name>   
    <value>hive</value>
      <description>username to use against metastore database</description>
      </property>
       
<property>
  <name>javax.jdo.option.ConnectionPassword</name>   
    <value>hive</value>
      <description>password to use against metastore database</description>
      </property>
吧
</configuration>



#### 1.25. Hive基本使用
1.创建示例表如下  
2.求每个部门人数 
`select deptno, count(1) from emp group by deptno;`



In [1]:
 
--创建emp表


create table emp
(empno int ,
ename string,
job string,
mgr int,
hiredate string,
sal double,
comm double,
deptno int 
) ROW FORMAT DELIMITED FIELDS TERMINATED BY '\t';

 
INSERT INTO emp VALUES(7369,'SMITH','CLERK',7902,'1980-12-17',800,NULL,20);
INSERT INTO emp VALUES(7499,'ALLEN','SALESMAN',7698,'1981-02-20',1600,300,30);
INSERT INTO emp VALUES(7521,'WARD','SALESMAN',7698,'1981-02-22',1250,500,30);
INSERT INTO emp VALUES(7566,'JONES','MANAGER',7839,'1981-04-02',2975,NULL,20);
INSERT INTO emp VALUES(7654,'MARTIN','SALESMAN',7698,'1981-09-28',1250,1400,30);
INSERT INTO emp VALUES(7698,'BLAKE','MANAGER',7839,'1981-05-01',2850,NULL,30);
INSERT INTO emp VALUES(7782,'CLARK','MANAGER',7839,'1981-06-09',2450,NULL,10);
INSERT INTO emp VALUES(7788,'SCOTT','ANALYST',7566,'1987-06-13',3000,NULL,20);
INSERT INTO emp VALUES(7839,'KING','PRESIDENT',NULL,'1981-11-17',5000,NULL,10);
INSERT INTO emp VALUES(7844,'TURNER','SALESMAN',7698,'1981-09-08',1500,0,30);
INSERT INTO emp VALUES(7876,'ADAMS','CLERK',7788,'1987-06-13',1100,NULL,20);
INSERT INTO emp VALUES(7900,'JAMES','CLERK',7698,'1981-12-03',950,NULL,30);
INSERT INTO emp VALUES(7902,'FORD','ANALYST',7566,'1981-12-03',3000,NULL,20);
INSERT INTO emp VALUES(7934,'MILLER','CLERK',7782,'1983-01-23',1300,NULL,10);
INSERT INTO emp VALUES(8888,'HIVE','PROGRAM',7839,'1988-01-23',10300,NULL);


--创建dept表

create table dept
(
deptno int  ,
dname string,
location string
)ROW FORMAT DELIMITED FIELDS TERMINATED BY '\t';



INSERT INTO dept VALUES(10,'ACCOUNTING','NEW YORK');
INSERT INTO dept VALUES(20,'RESEARCH','DALLAS');
INSERT INTO dept VALUES(30,'SALES','CHICAGO');
INSERT INTO dept VALUES(40,'OPERATIONS','BOSTON');

SyntaxError: invalid syntax (<ipython-input-1-32e5c66b27c0>, line 5)

<a id='2'></a>
# [第2章 Spark及其生态圈概述](#0)

 


https://spark.apache.org/
### 2.2. Spark 概述及特点
分布式计算框架
相比hadoop速度快很多
 <img src='https://spark.apache.org/images/logistic-regression.png' >
### 2.2. Spark 产生背景
* mapreduce的局限性
  * 代码繁琐
  * 只能支持map和reduce操作
  * 执行效率低下
  * 不适合迭代多次、交互式、流式的处理
* 框架多样化
  * 批处理（离线）：MapReduce，Hive，Pig
  * 流式处理（实时）：storm，jstom
  * 交互式计算：Impala   
  
> 框架多样化，导致需要多态集群，学习运维成本高

### 2.4. Spark发展历史
<img src='img/Spark发展历史.png'，width='66%'>
### 2.6. Spark 对比 Hadoop
### 2.7. 

<a id='3'></a>
# [第3章 实战环境搭建](#0)

### 3.1 Spark 源码编译
* 下载  wget https://archive.apache.org/dist/spark/spark-2.1.0/spark-2.1.0.tgz
* 解压  解压：tar -zxvf spark-2.1.0.tgz -C ~/app/
* 要求： Building Spark using Maven requires Maven 3.3.9 or newer and Java 7+.
* Maven3.3.9安装包下载 wget https://mirrors.tuna.tsinghua.edu.cn/apache//maven/maven-3/3.3.9/binaries/apache-maven-3.3.9-bin.tar.gz
* 解压： tar -zxvf apache-maven-3.3.9-bin.tar.gz -C ~/app/
* maven内存设置 export MAVEN_OPTS="-Xmx2g -XX:ReservedCodeCacheSize=512m"
* 编译命令
  * ./build/mvn -Pyarn -Phadoop-2.6 -Phive -Phive-thriftserver  -Dhadoop.version=2.6.0-cdh5.7.0 -DskipTests clean package

  * 推荐： 
  
  ./dev/make-distribution.sh --name 2.6.0-cdh5.7.0  --tgz -Pyarn -Phadoop-2.6 -Phive -Phive-thriftserver -Dhadoop.version=2.6.0-cdh5.7.0  -Pmesos 
  
  ./dev/make-distribution.sh --name 2.6.0-cdh5.7.0  --tgz -Pyarn -Phadoop-2.6 -Phive -Phive-thriftserver -Dhadoop.version=2.6.0-cdh5.7.0  -Pmesos 
  
    第二个编译完成后：spark-\$VERSINO-bin-\$NAME 



#### scala 安装配置哦
wgethttps://downloads.lightbend.com/scala/2.11.8/scala-2.11.8.tgz
解压：tar -zxvf scala-2.11.8.tgz -C ~/app/
设置Scala环境变量设置

   
验证Scala安装  

scala -version  
Scala code runner version 2.11.7 -- Copyright 2002-2013, LAMP/EPFL

In [2]:
 //将JDK目录添加到系统环境变量(~/.bash_profile)中 
export MAVEN_HOME=/home/hadoop/app/apache-maven-3.3.9 
export PATH=$MAVEN_HOME/bin:$PATH 
//让配置文件生效 source ~/.bash_profile
//执行mvn，查看版本
mvn -v 
//如果安装成功后，则有如下信息的输出 
Apache Maven 3.3.9 (bb52d8502b132ec0a5a3f4c09453c07478323dc5; 2015-11-10T08:41:47-08:00) 
Maven home: /home/hadoop/app/apache-maven-3.3.9 Java version: 1.7.0_51, vendor: Oracle Corporation 
Java home: /home/hadoop/app/jdk1.7.0_51/jre 
Default locale: zh_CN, platform encoding: UTF-8 
OS name: "linux", version: "2.6.32-358.el6.x86_64", arch: "amd64", family: "unix"
 

SyntaxError: invalid syntax (<ipython-input-2-63b03db53f5c>, line 1)

坑1：编译报错；
Failed to execute goal on project spark-launcher_2.11: Could not resolve dependencies for project org.apache.spark:spark-launcher_2.11:jar:2.1.0: Failure to find org.apache.hadoop:hadoop-client:jar:2.6.0-cdh5.7.0 in https://repo1.maven.org/maven2 was cached in the local repository, resolution will not be reattempted until the update interval of central has elapsed or updates are forced -> [Help 1]

坑2：虚拟机内存：2-4G
坑3： ./dev/change-scala-version.sh 2.11

坑4：zinc https://blog.csdn.net/jiaotangX/article/details/78635133

https://www.cloudera.com/documentation/enterprise/release-notes/topics/cdh_vd_cdh5_maven_repo.html  
https://spark.apache.org/docs/2.1.0/building-spark.html  
https://blog.csdn.net/wuzhilon88/article/details/73240571

坑：zinc https://blog.csdn.net/jiaotangX/article/details/78635133  
部分为修改项（此处是导致编译失败的根本原因）
```
<scalaVersion>${scala.version}</scalaVersion>
<recompileMode>incremental</recompileMode>
<useZincServer>false</useZincServer>

```

坑：
https://blog.csdn.net/wuzhilon88/article/details/73240571
http://www.ijava.com/article/hadoop-42617.html
https://blog.csdn.net/wuzhilon88/article/details/73240571
从失败的地方开始： 
https://blog.csdn.net/suisongtiao1799/article/details/80223068

In [1]:
# 报错，在pom.xml的<id>central</id>下插入

 
<repository>
    <id>cloudera</id>
       <url>https://repository.cloudera.com/artifactory/cloudera-repos/</url>
</repository>





<properties>
 
    <hadoop.version>2.2.0</hadoop.version>
    <protobuf.version>2.5.0</protobuf.version>
    <yarn.version>${hadoop.version}</yarn.version>
</properties>


<profile>
  <id>hadoop-2.6</id>
  <properties>
    <hadoop.version>2.6.4</hadoop.version>
    <jets3t.version>0.9.3</jets3t.version>
    <zookeeper.version>3.4.6</zookeeper.version>
    <curator.version>2.6.0</curator.version>
  </properties>
</profile>


SyntaxError: invalid syntax (<ipython-input-1-e2d5d8f90c5f>, line 4)

### 3.4. Spark Local模式环境搭建

* 解压： tar -zxvf spark-2.2.0-bin-2.6.0-cdh5.7.0.tgz -C ~/app/

* 删除cmd： rm -rf *.cmd 

```
export SPARK_HOME=/home/hadoop/app/spark-2.2.0-bin-2.6.0-cdh5.7.0 
export PATH=$SPARK_HOME/bin:$PATH

```
 `spark-shell --master local[2]`


### 3.5. Spark Standalone模式环境搭建
与HDFS/YARN很类似
1.master + n worker


[hadoop@master conf]$ `cp spark-env.sh.template spark-env.sh`


In [1]:

# Options for the daemons used in the standalone deploy mode

``` spark-env.sh
PARK_MASTER_HOST=master   
SPARK_WORKER_CORES=2
SPARK_WORKER_MEMORY=2g
SPARK_WORKER_INSTANCES=1
```

SyntaxError: invalid syntax (<ipython-input-1-e0a65de057de>, line 4)


http://www.voidcn.com/article/p-opptahtj-pb.html

 `SPARK_HOME/sbin/start-all.sh`
 

* 提示：JAVA_HOME is not set的错误提示。
https://blog.csdn.net/qq_27882063/article/details/58599276
在spark-env.sh追加  JAVA_HOME 相关参数
* 2:sbin目录一般与服务相关
 
如果：  
hadoop1:master  
hadoop2:worker  
hadoop2:worker  
然后配置：slaves.sh  
hadoop2  
hadoop3  
===》start-all.sh
会在master上启动master进程，在slaves文件配置的所以hostname的机器上启动worker进程
**启动**
`spark-shell --master spark://master:7077 `


### 3.6.spark 简单使用wordcount
wordcounts
```scala
val file=spark.sparkContext.textFile("file:///home/hadoop/data/wc.txt")
val wordCounts = file.flatMap(line => line.split(" ,")).map((word => (word,1))).reduceByKey(_ + _)
wordCounts.collect
```

<a id =4></a>
# [第4章 Spark SQL概述](#0)

### 4.1. 前世今生
关系型数据库  RDD DML

* Hive: 类似于sql的Hive QL语言，sql====>mapreduce
* Spark：hive on spark   ＝＝＝＝＞　ｓｈａｒｋ基于spark、基于内存的列式存储、与hive兼容。缺点：hive ql解析、逻辑执行。。。。。。依赖hive
* shark终止后： 1，hive on spark 2,Spark SQL
### 4.2. SQL on hadoop 框架
* Hive metastone:元数据 slq to mepreduce
* impala : cloudera开发（cdh，cm） sql
* presto: facebook开源（京东）sql

* drill:  sql 访问：hadf,rdbms json habse mangodb s3 hive

* Spark Sql: sql dataframe/dataset api
   * matastroen 
   * 访问：hadf,rdbms json habse mangodb s3 hive ===> 数据源
   
<img src='img/Spark_SQL架构.png' width='77%'>


<a id = 5></a>
# [第5章 从Hive平滑过渡到Spark SQL](#0)

### 5.1 SQLContext 的使用
> Spark 1.x中Spark　SQL的入口


```Scala
val sc: SparkContext // An existing SparkContext.
val sqlContext = new org.apache.spark.sql.SQLContext(sc)
```

### 5.2 HiveContext

### 5.4 SparkSession
> Spark 1.x中Spark　SQL的入口

```Scala
import org.apache.spark.sql.SparkSession

val spark = SparkSession
  .builder()
  .appName("Spark SQL basic example")
  .config("spark.some.config.option", "some-value")
  .getOrCreate()

// For implicit conversions like converting RDDs to DataFrames
import spark.implicits._

```

win 路径写法:`file:///D:/Data/people.json`

打jar包到客户端: `D:\Documents\Spark\SparkSQL>mvn clean package -DskipTests`



In [ ]:
# Launching Applications with spark-submit
./bin/spark-submit \
  --class <main-class> \
  --master <master-url> \
  --deploy-mode <deploy-mode> \
  --conf <key>=<value> \
  ... # other options
  <application-jar> \
  [application-arguments]
    
# examoles
# 提交
./bin/spark-submit \
 --class com.imooc.spark.SQLContextAPP \
 --master local[2] \
 /home/hadoop/lib/sql-1.0.jar \
/home/hadoop/app/spark-2.2.0-bin-2.6.0-cdh5.7.0/examples/src/main/resources/people.json

### 5.5 spark-shell 和spark-sql 的使用

#### spark-shell

 `spark-shell --master local[2]`
 
 * spark 控制台:   
 `spark.sql("show tables").show`  
 
* 为了在hive中读取数据,在 /home/hadoop/app/hive-1.1.0-cdh5.7.0/conf目录下:  
 `cp hive-site.xml ~/app/spark-2.2.0-bin-2.6.0-cdh5.7.0/conf/`
 
* 传入 jars包  
  `spark-shell --master local[2] --jars ~/software/mysql-connector-java-5.1.46-bin.jar`

之后可以查看数据库的数据

* 应用:

hive>
` select * from emp e join dept d on e.deptno=d.deptno;`


> Time taken: 43.799 seconds, Fetched: 14 row(s)


scala>
`spark.sql("select * from emp e join dept d on e.deptno=d.deptno").show`
> 立即出结果


* 处理报错ERROR
`/home/hadoop/app/spark-2.2.0-bin-2.6.0-cdh5.7.0/conf/hive-site.xml `
添加:
```
<property>
    <name>hive.metastore.schema.verification</name>
    <value>false</value>
</property>
```

#### spark-sql
* 开启:  
`spark-sql --master local[2] --jars ~/software/mysql-connector-java-5.1.46-bin.jar`

http://master:4040

spark-sql>  
` select * from emp e join dept d on e.deptno=d.deptno;`

* 创建表 查看执行计划
 
`create table t(key string, value string);`

`explain select a.key*(2+3), b.value  from t a join t b on a.key = b.key and a.key > 3;`

>  Physical Plan



* 查看详细计划,加上extended
`explain  extended select a.key*(2+3), b.value  from t a join t b on a.key = b.key and a.key > 3;`



In [ ]:
== Parsed Logical Plan ==
'Project [unresolvedalias(('a.key * (2 + 3)), None), 'b.value]
+- 'Join Inner, (('a.key = 'b.key) && ('a.key > 3))
   :- 'SubqueryAlias a
   :  +- 'UnresolvedRelation `t`
   +- 'SubqueryAlias b
      +- 'UnresolvedRelation `t`

== Analyzed Logical Plan ==
(CAST(key AS DOUBLE) * CAST((2 + 3) AS DOUBLE)): double, value: string
Project [(cast(key#25 as double) * cast((2 + 3) as double)) AS (CAST(key AS DOUBLE) * CAST((2 + 3) AS DOUBLE))#29, value#28]
+- Join Inner, ((key#25 = key#27) && (cast(key#25 as int) > 3))
   :- SubqueryAlias a
   :  +- SubqueryAlias t
   :     +- CatalogRelation `hive`.`t`, org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe, [key#25, value#26]
   +- SubqueryAlias b
      +- SubqueryAlias t
         +- CatalogRelation `hive`.`t`, org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe, [key#27, value#28]

== Optimized Logical Plan ==
Project [(cast(key#25 as double) * 5.0) AS (CAST(key AS DOUBLE) * CAST((2 + 3) AS DOUBLE))#29, value#28]
+- Join Inner, (key#25 = key#27)
   :- Project [key#25]
   :  +- Filter (isnotnull(key#25) && (cast(key#25 as int) > 3))
   :     +- CatalogRelation `hive`.`t`, org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe, [key#25, value#26]
   +- Filter (isnotnull(key#27) && (cast(key#27 as int) > 3))
      +- CatalogRelation `hive`.`t`, org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe, [key#27, value#28]

== Physical Plan ==
*Project [(cast(key#25 as double) * 5.0) AS (CAST(key AS DOUBLE) * CAST((2 + 3) AS DOUBLE))#29, value#28]
+- *SortMergeJoin [key#25], [key#27], Inner
   :- *Sort [key#25 ASC NULLS FIRST], false, 0
   :  +- Exchange hashpartitioning(key#25, 200)
   :     +- *Filter (isnotnull(key#25) && (cast(key#25 as int) > 3))
   :        +- HiveTableScan [key#25], CatalogRelation `hive`.`t`, org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe, [key#25, value#26]
   +- *Sort [key#27 ASC NULLS FIRST], false, 0
      +- Exchange hashpartitioning(key#27, 200)
         +- *Filter (isnotnull(key#27) && (cast(key#27 as int) > 3))
            +- HiveTableScan [key#27, value#28], CatalogRelation `hive`.`t`, org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe, [key#27, value#28]
Time taken: 0.405 seconds, Fetched 1 row(s)
18/07/14 09:44:28 INFO CliDriver: Time taken: 0.405 seconds, Fetched 1 row(s)

### 5.6. thriftserver/beeline的使用

* 启动thriftserver : 默认端口10000  
> /home/hadoop/app/spark-2.2.0-bin-2.6.0-cdh5.7.0/sbin

`./start-thriftserver.sh --master local[2] --jars ~/software/mysql-connector-java-5.1.46-bin.jar `

* 验证 `jps -m`  
SparkSubmit --master local[2]


* 启动beeline -u 指定地址 -n 当前机器名

` beeline -u jdbc:hive2://localhost:10000 -n hadoop`

注： thriftserver 和普通的spark-shell/spark-sql 区别
* spark-shell 和spark-sql都是一个spark application
* thriftserver, 可以启动多个客户端（beeline/cod)但是是一个spark application
 > 解决了一个数据共享的问题，多个客户端可以共享数据
 
 
### 5.7. JDBC方式编程访问


In [ ]:
 

package com.imooc.spark

import java.sql.DriverManager

/**
  * 通过JDBC的方式访问
  */
object SparkSQLThriftServerApp {
  def main(args: Array[String]): Unit = {

    Class.forName("org.apache.hive.jdbc.HiveDriver")

    val conn = DriverManager.getConnection("jdbc:hive2://master:10000","hadoop","")
    val stmt = conn.prepareStatement("use hive")
    val rst = stmt.executeQuery()
    val pstmt = conn.prepareStatement("select empno, ename, sal from emp")
    val rs = pstmt.executeQuery()
    while (rs.next()){
      println("empno"+rs.getInt("empno")+
        ",ename"+rs.getString("ename")+
        ",sal:"+rs.getDouble("sal"))
    }
    rs.close()
    pstmt.close()
    conn.close()
  }
}
 

<a id = "6"></a>
# [第6章 DateFrame&Dataset](#0)

### 6.2. DataFrame 产生背景

java/scala ----> R/pandas

### 6.3. DataFrame 概述
- A Dataset is a distributed collection of data. A DataFrame is a Dataset organized into named columns.(RDD with schema)
> 以列的形式构成的分布式数据集，按照列赋予不同的名称

- It is conceptually equivalent to a **table** in a relational database or a data frame in R/Python
- An abstraction for selecting, filtering , aggregation and plotting structured data
- Inspired by R and Pandas single machine small data processing experiences applied to distributed big data, i.e.
### 6.4.DataFrame和RDD的对比
<img src="./img/DataFrame和RDD对比.png ", width = "66%">


* RDD
    - java/scala ==> jvm 
    - python ==> python runtime
* DataFrame 
    - java/scala/python ==> Login Plan
    
### 6.5. DataFrame基本API操作


``` scala
package com.imooc.spark

import org.apache.spark.sql.SparkSession


/**
  * DataFrame API基本操作
  */
object DataFrameApp {

  def main(args: Array[String]): Unit = {

    val spark  = SparkSession.builder().appName("DataFrameApp").master("local[2]").getOrCreate()
    //将json文件加载成DataFrame
    val peopleDF = spark.read.format("json").load("file:///D:/Data/people.json")
    // 输打DataFrame的schema信息
    peopleDF.printSchema()

    // 输出信息，默认20条
    // peopleDF.show()
    // 查询某列
    peopleDF.select("name").show()

    peopleDF.select(peopleDF.col("name"), peopleDF.col("age") +10).show()

    peopleDF.filter(peopleDF.col("age")>29).show()

    // 根据某一列进行分组，然后再进行聚合操作： select age.count(1) from table group by age

    peopleDF.groupBy("age").count().show()
    spark.stop()
    
  }
}
```

### 6.6. DataFrame与RDD互操作方式

Spark SQL supports two different methods for converting existing RDDs into Datasets.
- The first method uses **reflection** to infer the schema of an RDD that contains specific types of objects. This reflection based approach leads to more concise code and works well when you already know the schema while writing your Spark application.

- The second method for creating Datasets is through a programmatic interface that allows you to construct a schema and then apply it to an existing RDD. While this method is more verbose, it allows you to construct Datasets when the columns and their types are not known until runtime.

 #### 1.反射
 


``` scala
package com.imooc.spark
import org.apache.spark.sql.SparkSession

/**
  * DataFrame和RDD的互操作
 */
object DataFrameRDDApp {
  def main(args: Array[String]): Unit = {
    val spark = SparkSession.builder().appName("DataFrameRDDApp").master("local[2]").getOrCreate()
    // RDD 转换为 DataFrame
    // 1.获取RDD
    val rdd = spark.sparkContext.textFile("file:///D:/Data/infos.txt")
    // map 将获取的RDD每一行用“，”分割,分割后的字符串转换格式
    import spark.implicits._  // 隐式转换
    val infoDF = rdd.map(_.split(",")).map(line => Info(line(0).toInt,line(1),line(2).toInt)).toDF()

    infoDF.show()

    infoDF.filter(infoDF.col("age") > 30).show()

    // 注册成临时表，可用sql直接操作
    infoDF.createOrReplaceTempView("infos")

    spark.sql("select * from  infos where age > 30").show()

    spark.stop()
  }
  // 反射借助case class Info: 2,lisi,30
  case class Info(id:Int, name:String, age:Int)

}
```

#### 2. 编程方式

<a id = "7"></a>
# [第7章 External Data Source](#0)

方便从不同数据源，（json,parquet,rdbms),经过混合处理（json join parquet),再将处理结果以特定的格式（json，parquet）写回到指定的系统（HDFS。S3） 上去

产生：
Spark SQL 1.2 ==> 外部数据源API

#### 产生背景：
- Loading and saving Data is not easy
- parse raw data: text/json/parquet
- Conver data format transformation
- Datasets stored in various Formats/Systems

#### 目标
- Developer： build libraties for various data sources 
- User: easy loading/saving DataFrames



<a id = "9"></a>
# [第9章 慕课网日志实战](#0)

### 9.1.课程目录
- 用户行为日志概述
- 离线数据处理架构
- 项目需求
- 功能实现
- Spark on YARN（目前大多数公司的办法）
- 性能调优（重要，特别是面试）


### 9.2.用户行为概述
用户行为日志：用户每次访问网站时所有的行为数据(访问、浏览、搜索、点击....)

- 为什么记录用户访问行为日志
    - 网站页面的访问量
    - 网站的黏性
    - 推荐
  
- 用户行为日志生成渠道
    - Nginx
    - Ajax
    
- 用户行为日志内容
    - 系统属性：操作系统、浏览器
    - 访问特征：点击的 url、从哪个url跳转过来的（referer）、页面停留时间
    - 访问信息： session_id、访问ip等
 
- 用户行为日志分析的意义
    - 网站的眼睛
    - 网站的神经
    - 网站的大脑
    

     
   
    

### 9.3.离线数据处理架构
<img src = "./img/离线数据处理架构.png" width = "88%">
- 数据采集  
Flume： web日志写入到HDFS


- 数据清洗  
处理脏数据  
Spark、Hive、MapReduce（或者是其他分布式计算框架）
清洗完后存放到HDFS（hive/Spark SQL）


- 数据处理  
按照需求进行相应业务的统计和分析
Spark、Hive、MapReduce（或者是其他分布式计算框架）


- 处理结果入库  
结果可以存放到RDBMS、NoSQL    


- 数据可视化  
通过图形化方式展示：饼图，柱状图，折线图  
ECharts、HUE、Zeppelin

### 9.3.项目需求

- 统计主站最受欢迎的课程/手记的Top N访问次数


- 按地市统计主站最受欢迎的Top N课程   
    根据IP地址提取出地市信息，窗口函数在Spark SQL中的使用
    
    
- 按照流量统计最受欢迎的TopN课程